# Generating Song Lyrics Using RNN

We have learned enough about RNNs; now, we will look at how to generate song lyrics using RNNs. To do this, we simply build a character-level RNN, meaning that on every time step, we predict a new character.

Let's consider a small sentence: _"What a beautiful d."_ 

At the first time step, the RNN predicts a new character as a. The sentence will be updated to, _"What a beautiful da."_

At the next time step, it predicts a new character as y, and the sentence becomes, _"What a beautiful day."_

In this manner, we predict a new character at each time step and generate a song. Instead of predicting a new character every time, we can also predict a new word every time, which is called word level RNN. For simplicity, let's start with a character level RNN.

How do RNNs predict a new character on each time step? Let's suppose that at a time step t=0, we feed an input character, let's say, x. Now, the RNN predicts the next character based on the given input character, x. To predict the next character, it predicts the probability of all the characters in our vocabulary to be the next character. Once we have this probability distribution, we randomly select the next character based on this probability. Confusing? Let us better understand this with an example. 

For instance, as shown in the following figure, let's suppose that our vocabulary contains four characters L, O, V, and E; when we feed the character l as an input, it computes the probability of all the words in the vocabulary to be the next character:


![image](images/2.png)


So, we have the probabilities as [0.0, 0.9, 0.0, 0.1], corresponding to the characters in the vocabulary [L,O,V,E]. With this probability distribution, we select O as the next character 90% of the time, and E as the next character 10% of the time. Predicting the next character by sampling from this probability distribution adds some randomness to the output. 

On the next time step, we feed the predicted character from the previous time step and the previous hidden state as an input to predict the next character, as shown in the following figure:


![image](images/3.png)

So, on each time step, we feed the predicted character from the previous time step and the previous hidden state as input and predict the next character:


![image](images/4.png)

As you can see in the above figure, at time step t=2, V is passed as an input, and it predicts the next character as E. But this does not mean that every time character V is sent as an input it should always return E as output. Since we are passing input along with the previous hidden state, the RNN has the memory of all the characters it has seen so far.

So, the previous hidden state captures the essence of the previous input characters, which are L and O. Now, with this previous hidden state and the input V, the RNN predicts the next character as E.

## Implementing RNN in TensorFlow

Now, we will look at how to build the RNN model in TensorFlow to generate song lyrics. The dataset used in this section available in the data folder as songdata.zip. You can unzip the dataset and place it in the data folder and start running the below code.

First, let us import the required libraries:

In [1]:
import warnings
warnings.filterwarnings('ignore')

import random
import numpy as np
import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.ERROR)

## Data Preparation

Read the downloaded input dataset:

In [2]:
df = pd.read_csv('data/songdata.csv')

Let us see few rows from our data:

In [3]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


Our dataset consists of about 57,650 song lyrics:

In [4]:
df.shape[0]

57650

We have song lyrics of about 643 artists:

In [5]:
len(df['artist'].unique())

643

The number of songs from each artist is shown as follows:

In [6]:
df['artist'].value_counts()[:10]

Donna Summer        191
Gordon Lightfoot    189
George Strait       188
Bob Dylan           188
Loretta Lynn        187
Cher                187
Alabama             187
Reba Mcentire       187
Chaka Khan          186
Dean Martin         186
Name: artist, dtype: int64

On average, we have about 89 songs of each artist:

In [7]:
df['artist'].value_counts().values.mean()

89.65785381026438

We have song lyrics in the column text, so we combine all the rows of that column and save it as a text in a variable called data, as follows:

In [8]:
data = ', '.join(df['text'])

Let's see a few lines of a song:

In [9]:
data[:369]

"Look at her face, it's a wonderful face  \nAnd it means something special to me  \nLook at the way that she smiles when she sees me  \nHow lucky can one fellow be?  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what co"

Since we are building a char-level RNN, we will store all the unique characters in our dataset into a variable called chars; this is basically our vocabulary:

In [10]:
chars = sorted(list(set(data)))


Store the vocabulary size in a variable called vocab_size:

In [11]:
vocab_size = len(chars)


Since the neural networks only accept the input in numbers, we need to convert all the characters in the vocabulary to a number.

We map all the characters in the vocabulary to their corresponding index that forms a unique number. We define a char_to_ix dictionary, which has a mapping of all the characters to the index. To get the index by a character, we also define the ix_to_char dictionary, which has a mapping of all the indices to their respective characters:

In [12]:
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

As you can see in the following code snippet, the character 's' is mapped to an index 68 in the char_to_ix dictionary:

In [13]:
char_to_ix['s']

68

Similarly, if we give 68 as an input to the ix_to_char, then we get the corresponding character, which is 's':

In [14]:
ix_to_char[68]

's'


Once we obtain the character to integer mapping, we use one-hot encoding to represent the input and output in vector form. A one-hot encoded vector is basically a vector full of 0s, except for a 1 at a position corresponding to a character index.

For example, let's suppose that the vocabSize is 7, and the character z is in the fourth position in the vocabulary. Then, the one-hot encoded representation for the character z can be represented as follows:

In [15]:
vocabSize = 7
char_index = 4

np.eye(vocabSize)[char_index]

array([0., 0., 0., 0., 1., 0., 0.])

As you can see, we have a 1 at the corresponding index of the character, and the rest of the values are 0s. This is how we convert each character into a one-hot encoded vector. 

In the following code, we define a function called one_hot_encoder, which will return the one-hot encoded vectors, given an index of the character:

In [16]:
def one_hot_encoder(index):
    return np.eye(vocab_size)[index]

## Defining the Network Parameters


We need to define all the network parameters. 

In [17]:
#define the number of units in the hidden layer:
hidden_size = 100  
 
#define the length of the input and output sequence:
seq_length = 25  

#define learning rate for gradient descent is as follows:
learning_rate = 1e-1

#set the seed value:
seed_value = 42
tf.set_random_seed(seed_value)
random.seed(seed_value)

## Defining Placeholders

Now, we will define the TensorFlow placeholders. The placeholders for the input and output are as follows:

In [18]:
inputs = tf.placeholder(shape=[None, vocab_size],dtype=tf.float32, name="inputs")
targets = tf.placeholder(shape=[None, vocab_size], dtype=tf.float32, name="targets")

Define the placeholder for the initial hidden state:

In [19]:
init_state = tf.placeholder(shape=[1, hidden_size], dtype=tf.float32, name="state")

Define an initializer for initializing the weights of the RNN:

In [20]:
initializer = tf.random_normal_initializer(stddev=0.1)

## Defining forward propagation

Let's define the forward propagation involved in the RNN, which is mathematically given as follows:

$$ h_t =  \operatorname{tanh}(U x_t + W h_{t-1} + bh) $$
$$ \hat{y} =  \operatorname{softmax}(V h_t + by) $$

In [21]:
with tf.variable_scope("RNN") as scope:
    h_t = init_state
    y_hat = []

    for t, x_t in enumerate(tf.split(inputs, seq_length, axis=0)):
        if t > 0:
            scope.reuse_variables()  

        #input to hidden layer weights
        U = tf.get_variable("U", [vocab_size, hidden_size], initializer=initializer)

        #hidden to hidden layer weights
        W = tf.get_variable("W", [hidden_size, hidden_size], initializer=initializer)

        #output to hidden layer weights
        V = tf.get_variable("V", [hidden_size, vocab_size], initializer=initializer)

        #bias for hidden layer
        bh = tf.get_variable("bh", [hidden_size], initializer=initializer)

        #bias for output layer
        by = tf.get_variable("by", [vocab_size], initializer=initializer)

        h_t = tf.tanh(tf.matmul(x_t, U) + tf.matmul(h_t, W) + bh)

        y_hat_t = tf.matmul(h_t, V) + by

        y_hat.append(y_hat_t)       

Apply softmax on the output and get the probabilities:

In [22]:
output_softmax = tf.nn.softmax(y_hat[-1])  

outputs = tf.concat(y_hat, axis=0)

Compute the cross-entropy loss:

In [23]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=outputs))

Store the final hidden state of the RNN in hprev. We use this final hidden state for making predictions:

In [24]:
hprev = h_t

## Defining Backpropagation Through Time

Now, we will perform the BPTT, with Adam as our optimizer. We will also perform gradient clipping to avoid the exploding gradients problem.

Initialize the Adam optimizer:

In [25]:
minimizer = tf.train.AdamOptimizer()

Compute the gradients of the loss with the Adam optimizer:

In [26]:
gradients = minimizer.compute_gradients(loss)

Set the threshold for the gradient clipping:

In [27]:
threshold = tf.constant(5.0, name="grad_clipping")

Clip the gradients which exceeds the threshold and bring it to the range:

In [28]:
clipped_gradients = []
for grad, var in gradients:
    clipped_grad = tf.clip_by_value(grad, -threshold, threshold)
    clipped_gradients.append((clipped_grad, var))

Update the gradients with the clipped gradients:

In [29]:
updated_gradients = minimizer.apply_gradients(clipped_gradients)

## Start generating songs 



Start the TensorFlow session and initialize all the variables:

In [30]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

## Step by Step Explanation

__First let us understand how RNN is generating song lyrics step by step, the complete code is given at the end of the notebook. __

Now, we will look at how to generate the song lyrics using an RNN. What should the input and output to the RNN be? How does it learn? What is the training data? Let's see the explanation, along with the code, step by step. 

We know that in RNNs, the output predicted at a time step $t$ will be sent as the input to the next time step; that is, on every time step, we need to feed the predicted character from the previous time step as input. So, we prepare our dataset in the same way. 

For instance, look at the following table. Let's suppose that each row is a different time step; on a time step $t=0$ , the RNN predicted a new character, g, as the output. This will be sent as the input to the next time step $t=1$, . 

However, if you notice the input in the time step $t=1$, we removed the first character from the input o and added the newly predicted character g at the end of our sequence. Why are we removing the first character from the input? Because we need to maintain the sequence length. 

Let's suppose that our sequence length is eight; adding a newly predicted character to our sequence increases the sequence length to nine. To avoid this, we remove the first character from the input, while adding a newly predicted character from the previous time step.

Similarly, in the output data, we also remove the first character on each time step, because once it predicts the new character, the sequence length increases. To avoid this, we remove the first character from the output on each time step, as shown in the following table: 

![image](images/5.png)

Now, we will look at how we can prepare our input and output sequence similar to the preceding table. __The code is bolded. __

Define a variable called pointer, which points to the character in our dataset. We will set our pointer to 0, which means it points to the first character:


__pointer = 0__

Define the input data:

__input_sentence = data[pointer: pointer + seq_length]__


What does this mean? With the pointer and the sequence length, we slice the data. Consider that the seq_length is 25 and the pointer is 0. It will return the first 25 characters as input. So, data[pointer:pointer + seq_length] returns the following output:

_"Look at her face, it's a "_


Define the output, as follows:

__output_sentence = data[pointer + 1: pointer + seq_length + 1]__


We slice the output data with one character ahead moved from input data. So, data[pointer + 1:pointer + seq_length + 1] returns the following:

_"ook at her face, it's a w"_



As you can see, we added the next character in the sentence and removed the first character. So, on every iteration, we increment the pointer and traverse the entire dataset. This is how we obtain the input and output sentence for training the RNN. 

As you have learned, an RNN only accepts numbers as input. Thus, once we sliced the input and output sequence, we get the indices of the respective characters, using the char_to_ix dictionary that we defined:

__input_indices = [char_to_ix[ch] for ch in input_sentence]__

__target_indices = [char_to_ix[ch] for ch in output_sentence]__ 



Convert the indices into one-hot encoded vectors by using the one_hot_encoder function we defined previously:

__input_vector = one_hot_encoder(input_indices)__

__target_vector = one_hot_encoder(target_indices)__


This input_vector and target_vector become the input and output for training the RNN. Let's start training.

The hprev_val variable stores the last hidden state of our trained RNN model. We use this for making predictions, and we store the loss in loss_val:

__hprev_val, loss_val, _ = sess.run([hprev, loss, updated_gradients], feed_dict={inputs: input_vector,targets: target_vector,init_state: hprev_val})__



We train the model for n iterations. After training, we start making predictions. Now, we will look at how to make predictions and generate song lyrics using our trained RNN. Set the sample_length, that is, the length of the sentence (song) we want to generate:

__sample_length = 500__

Randomly select the starting index of the input sequence:

__random_index = random.randint(0, len(data) - seq_length)__ 

Select the input sentence with the randomly selected index:

__sample_input_sent = data[random_index:random_index + seq_length]__

As we know, we need to feed the input as numbers; convert the selected input sentence to indices:

__sample_input_indices = [char_to_ix[ch] for ch in sample_input_sent]__

Remember, we stored the last hidden state of the RNN in hprev_val. We used that for making predictions. Now, we will create a new variable called sample_prev_state_val by copying values from hprev_val.

The sample_prev_state_val variable is used as an initial hidden state for making predictions:

__sample_prev_state_val = np.copy(hprev_val)__


Initialize the list for storing the predicted output indices:

__predicted_indices = []__

Now, for t in range of sample_length, we perform the following and generate the song for the defined sample_length. 

Convert the sampled_input_indices to the one-hot encoded vectors:

__sample_input_vector = one_hot_encoder(sample_input_indices)__

Feed the sample_input_vector, and also the hidden state sample_prev_state_val, as the initial hidden state to the RNN, and get the predictions. We store the output probability distribution in probs_dist:

__probs_dist, sample_prev_state_val = sess.run([output_softmax, hprev],
 feed_dict={inputs: sample_input_vector,init_state: sample_prev_state_val})__
 
Randomly select the index of the next character with the probability distribution generated by the RNN:

__ix = np.random.choice(range(vocab_size), p=probs_dist.ravel())__

Add this newly predicted index, ix, to the sample_input_indices, and also remove the first index from sample_input_indices to maintain the sequence length. This will form the input for the next time step:

__sample_input_indices = sample_input_indices[1:] + [ix]__

Store all the predicted chars indices in the predicted_indices list:

__predicted_indices.append(ix)__

Convert all the predicted_indices to their characters:

__predicted_chars = [ix_to_char[ix] for ix in predicted_indices]__

Combine all the predicted_Chars and save it as text:

__text = ''.join(predicted_Chars)__

Print the predicted text on every 50,000th  iteration:

__print ('\n')__

__print (' After %d iterations' %(iteration))__

__print('\n %s \n' % (text,))__

__print('-'*115)__


Increment the pointer and iteration:

__pointer += seq_length__
__iteration += 1__

## Complete code block for generating songs

In [31]:
pointer = 0
iteration = 0

In [ ]:
while True:
    
    if pointer + seq_length+1 >= len(data) or iteration == 0:
        hprev_val = np.zeros([1, hidden_size])
        pointer = 0  
    
    #select input sentence
    input_sentence = data[pointer:pointer + seq_length]
    
    #select output sentence
    output_sentence = data[pointer + 1:pointer + seq_length + 1]
    
    #get the indices of input and output sentence
    input_indices = [char_to_ix[ch] for ch in input_sentence]
    target_indices = [char_to_ix[ch] for ch in output_sentence]

    #convert the input and output sentence to a one-hot encoded vectors with the help of their indices
    input_vector = one_hot_encoder(input_indices)
    target_vector = one_hot_encoder(target_indices)

    
    #train the network and get the final hidden state
    hprev_val, loss_val, _ = sess.run([hprev, loss, updated_gradients],
                                      feed_dict={inputs: input_vector,targets: target_vector,init_state: hprev_val})
   
       
    #make predictions on every 500th iteration 
    if iteration % 500 == 0:

        #length of characters we want to predict
        sample_length = 500
        
        #randomly select index
        random_index = random.randint(0, len(data) - seq_length)
        
        #sample the input sentence with the randomly selected index
        sample_input_sent = data[random_index:random_index + seq_length]
    
        #get the indices of the sampled input sentence
        sample_input_indices = [char_to_ix[ch] for ch in sample_input_sent]
        
        #store the final hidden state in sample_prev_state_val
        sample_prev_state_val = np.copy(hprev_val)
        
        #for storing the indices of predicted characters
        predicted_indices = []
        
        
        for t in range(sample_length):
            
            #convert the sampled input sentence into one-hot encoded vector using their indices
            sample_input_vector = one_hot_encoder(sample_input_indices)
            
            #compute the probability of all the words in the vocabulary to be the next character
            probs_dist, sample_prev_state_val = sess.run([output_softmax, hprev],
                                                      feed_dict={inputs: sample_input_vector,init_state: sample_prev_state_val})

            #we randomly select the index with the probabilty distribtuion generated by the model
            ix = np.random.choice(range(vocab_size), p=probs_dist.ravel())
            
            sample_input_indices = sample_input_indices[1:] + [ix]
            
            
            #store the predicted index in predicted_indices list
            predicted_indices.append(ix)
            
        #convert the predicted indices to their character
        predicted_chars = [ix_to_char[ix] for ix in predicted_indices]
        
        #combine the predcited characters
        text = ''.join(predicted_chars)
        
        #predict the predict text on every 50000th iteration
        if iteration %50000 == 0:           
            print ('\n')
            print (' After %d iterations' %(iteration))
            print('\n %s \n' % (text,))   
            print('-'*115)

            
    #increment the pointer and iteration
    pointer += seq_length
    iteration += 1



 After 0 iterations

 uhiq iUYo9ra)1FZTezk3FGF CoG)i,uzWKrmpmqJuo(94rKty5"y-7t,1])zfwR2 FhFHcqdP2qy[!mw(1R[?xS?n(O-"x5?"k!efK MCnHSNA0h!SovpSppQ-(m,KBfn9"j.95p86F?Mun0[qdJ-L7F.Wv!W.GunT9CnwfGobu"WA?qAtfmREbZGCjDYvl:jN"D7?iMisuv2hgH1((Z0XepXA7G.Z28znn'SaEzDzCUuM7Fr0)ahqaX7!sJf-B1a5gO!iW8LbnJO1q" kjB jI2ZNd F-AX-hHhS-rM?RMy73ON[(Xu,N3T[AYD[?anzGVaBn4A Un0fOt!aDdNt7C)dSYapFz[!79"B9Z5Y!KJLD (zQDZmjpnY7-546t?T ?shi8W BA8Yp.ghO7up[:p?zM7::tuU "QDy97iWi:B:YBCVZ4)7NJnO[OW3zMvzc4liEs'CY,6-M,B[hBZ ?wtxdCRi-GlXtc[TmaB07L2Ui!hkk 

-------------------------------------------------------------------------------------------------------------------


 After 50000 iterations

 davest:]  
[Brike and ener the clifes  
Surs-  
[Erd ring of me hroutse all what you  
Y shrdie to thesp a Whone arive sraie  
Take it herne  
Mrme?  
I come me hect try love:]  
Onever love  
What lefter. pripls  
  
[Vny Mvary-as in everythes arffre?  
[Drel this Mry a claze  
And drizes.  
  
[Ars.  
Sark:]  
Er a dide a can

After training over several iterations, RNN will learn to generate better songs. In order to get better results, you can train the network with huge dataset for several iterations. 



In the next section, we will learn about the several different types of RNN architectures.